In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

## Preparing data in the format for ner tags

Original input data is in format :
* Each input article was given in seperated text file (article id in text file name), for train and dev sets
* With the same article id in label file , we get annotation of propaganda spans:
    * article_id propaganda start_span end_span

    * article id 1111 is present in article_1111.txt and label is present in article_1111_TC.txt 
         * article_1111.txt : Trump the white president say he likes black people working from him .
         * article_1111_TC.txt : 
             * 1111 <propaganda type> 11 88

    
From this input we convert we covert the articles to files for each article ids , but in positions of span text , markings of start span and end span.This is done by using the scripts given by the competition organizer. For example : 

 * the output after processing looks like 
    
    article_1111.txt : Trump the <span-7> white president say he likes black people working from him. <7-/span> 
    
    the id 7 comes from type of propaganda. each propaganda has been given a seperated id .
    
From the above output the following preprocessing turns this article id into seperate sentences each having ner like labels
    
Example 
    
      file no : article_1111.txt
      
      Trump O
      the O
      white I 
      president I
      say I
      he I
      likes I
      black I
      people I
      working I
      from I
      him I
      . I


In [2]:
import pandas as pd
import glob
import swifter
path = r'/data/semeval-2020/task-11/datasets/train-tagged_article/'
all_files = glob.glob(path + "/*.txt")

content_tuple = []

for filename in all_files:
    file = open(filename)
    content = "".join(file.readlines()).replace("\n"," ")
    article_id = filename.split("article")[-1].split(".txt")[0]
    content_tuple.append((article_id,content))

In [3]:
content_df = pd.DataFrame(content_tuple,columns=["article_id","content"])

In [4]:
content_df.head(3)

article_id  \
0  762956953   
1  787529309   
2  999001296   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [5]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English   # updated , 
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
tokenizer = Tokenizer(nlp.vocab)

content_df[content_df["article_id"] == "735855251"].content[30]

'ICE arrests 20 in Kansas City during 4-day operation targeting criminal aliens  KANSAS CITY, Mo. — Federal officers with U.S. Immigration and Customs Enforcement’s (ICE) Enforcement and Removal Operations (ERO) arrested 20 criminal aliens and immigration violators in the Kansas City metro area during a four-day enforcement operation, which ended Thursday. During this operation, ERO deportation officers made arrests in the following Missouri cities: St. Joseph (6), Belton (1), Blue Springs (1), Independence (2) and Kansas City (6). ICE officers also made arrests in the Kansas cities of Olathe (3) and Lawrence (1). Fifteen men and five women, ages ranging 18-61, were arrested. Aliens arrested during this operation are from the following countries: Brazil (1), El Salvador (3), Guatemala (6), Honduras (1), Mexico (7), Romania (1) and Sierra Leone (1). Several of the aliens targeted by ERO deportation officers during this operation had prior criminal histories that included driving under t

In [6]:
for i in nlp(content_df[content_df["article_id"] == "735855251"].content[30]).sents:
    print(i)
    print("--")

ICE arrests 20 in Kansas City during 4-day operation targeting criminal aliens  KANSAS CITY, Mo. — Federal officers with U.S. Immigration and Customs Enforcement’s (ICE) Enforcement and Removal Operations (ERO) arrested 20 criminal aliens and immigration violators in the Kansas City metro area during a four-day enforcement operation, which ended Thursday.
--
During this operation, ERO deportation officers made arrests in the following Missouri cities: St. Joseph (6), Belton (1), Blue Springs (1), Independence (2) and Kansas City (6).
--
ICE officers also made arrests in the Kansas cities of Olathe (3) and Lawrence (1).
--
Fifteen men and five women, ages ranging 18-61, were arrested.
--
Aliens arrested during this operation are from the following countries: Brazil (1), El Salvador (3), Guatemala (6), Honduras (1), Mexico (7), Romania (1) and Sierra Leone (1).
--
Several of the aliens targeted by ERO deportation officers during this operation had prior criminal histories that included d

In [7]:
def split_into_sentences(row):
    sentences = []
    for ix,sent in enumerate(nlp(row.content).sents):
        sentences.append((row.article_id,ix,sent.text))
    return sentences    

In [8]:
sentences_dataset = content_df.apply(split_into_sentences,axis=1).values.tolist()

In [9]:
len(sentences_dataset)

371

In [10]:
sentences_dataset = [j for i in sentences_dataset for j in i]

In [11]:
len(sentences_dataset)

15565

In [12]:
sentences_dataset[:10]

[('762956953',
  0,
  'Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks  This has long been known, although the mainstream media dismissed it as a conspiracy theory.'),
 ('762956953', 1, 'But now we have definitive confirmation.'),
 ('762956953',
  2,
  'It was Iran Bush should have invaded after 9/11 , not Iraq.'),
 ('762956953',
  3,
  'Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.'),
 ('762956953',
  4,
  'The Iran nuclear deal should never have proceeded — President Obama, <span-6 the worst president in American history 6-/span> . “'),
 ('762956953',
  5,
  'Iran Admits To Facilitating 9/11 Terror Attacks,” by Adam Kredo, Washington Free Beacon, June 8, 2018: Iranian officials, in a first, have admitted to facilitat

In [13]:
sentences_df = pd.DataFrame(sentences_dataset,columns=["article_id","sentence_id","sentence"])

In [14]:
sentences_df.head(4)

,article_id,sentence_id,sentence
0,762956953,0,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory."
1,762956953,1,But now we have definitive confirmation.
2,762956953,2,"It was Iran Bush should have invaded after 9/11 , not Iraq."
3,762956953,3,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic."


In [15]:
sentences_df[sentences_df["article_id"] == "735855251"]

,article_id,sentence_id,sentence
1049,735855251,0,"ICE arrests 20 in Kansas City during 4-day operation targeting criminal aliens KANSAS CITY, Mo. — Federal officers with U.S. Immigration and Customs Enforcement’s (ICE) Enforcement and Removal Operations (ERO) arrested 20 criminal aliens and immigration violators in the Kansas City metro area during a four-day enforcement operation, which ended Thursday."
1050,735855251,1,"During this operation, ERO deportation officers made arrests in the following Missouri cities: St. Joseph (6), Belton (1), Blue Springs (1), Independence (2) and Kansas City (6)."
1051,735855251,2,ICE officers also made arrests in the Kansas cities of Olathe (3) and Lawrence (1).
1052,735855251,3,"Fifteen men and five women, ages ranging 18-61, were arrested."
1053,735855251,4,"Aliens arrested during this operation are from the following countries: Brazil (1), El Salvador (3), Guatemala (6), Honduras (1), Mexico (7), Romania (1) and Sierra Leone (1)."
1054,735855251,5,"Several of the aliens targeted by ERO deportation officers during this operation had prior criminal histories that included driving under the influence, child neglect, child abuse, drug offenses, fraud and larceny."
1055,735855251,6,"Four of these were arrested for illegally re-entering the United States after having been deported, which is a felony."
1056,735855251,7,Two overstayed lawful visits to the U.S. All were amenable to arrest and removal under the U.S. Immigration and Nationality Act.
1057,735855251,8,"The following are criminal summaries of some of the offenders arrested in the Kansas City area during this operation: A 55-year-old, Mexican citizen who overstayed a lawful visit to the U.S. by more than 12 years."
1058,735855251,9,"She was arrested Feb. 26, 2018 in Johnson County Kansas."


In [16]:
"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory.".find("span")

-1

In [17]:
def has_propaganda(sentence):
    if sentence.find("-/span") > 0:
        return True
    if sentence.find("span-") > 0:
        return True
    return False

In [18]:
sentences_df["has_propaganda"] = sentences_df["sentence"].apply(has_propaganda)

In [19]:
sentences_df.head(4)

,article_id,sentence_id,sentence,has_propaganda
0,762956953,0,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory.",False
1,762956953,1,But now we have definitive confirmation.,False
2,762956953,2,"It was Iran Bush should have invaded after 9/11 , not Iraq.",False
3,762956953,3,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.",False


In [20]:
sentences_df.has_propaganda.value_counts()

False    10608
True     4957 
Name: has_propaganda, dtype: int64

In [21]:
import re
def cleaned_sentence(sentence):
    return re.sub("<span-\d+", "", re.sub("\d+-/span>", "", sentence))

In [22]:
sentences_df["cleaned_sentence"] = sentences_df["sentence"].apply(cleaned_sentence)

In [23]:
sentences_df.head()

,article_id,sentence_id,sentence,has_propaganda,cleaned_sentence
0,762956953,0,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory.",False,"Iran Admits To Aiding Al-Qaeda and Facilitating 9/11 Jihad Terror Attacks This has long been known, although the mainstream media dismissed it as a conspiracy theory."
1,762956953,1,But now we have definitive confirmation.,False,But now we have definitive confirmation.
2,762956953,2,"It was Iran Bush should have invaded after 9/11 , not Iraq.",False,"It was Iran Bush should have invaded after 9/11 , not Iraq."
3,762956953,3,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic.",False,"Now consider this: even though, as President of the United States, Barack Obama had access to information that the general public does not have, and certainly knew of Iran’s involvement in the 9/11 attacks, Obama still pursued the Iran nuclear deal and gave billions to the Islamic Republic."
4,762956953,4,"The Iran nuclear deal should never have proceeded — President Obama, <span-6 the worst president in American history 6-/span> . “",True,"The Iran nuclear deal should never have proceeded — President Obama, the worst president in American history . “"


In [24]:
len(sentences_df)

15565

In [25]:
sentences_df[sentences_df["cleaned_sentence"] == ""]

,article_id,sentence_id,sentence,has_propaganda,cleaned_sentence
1068,735855251,19,6-/span>,True,
1656,697959084,19,5-/span>,True,
1802,999000147,24,5-/span>,True,
7750,721890296,43,8-/span>,True,
10547,705035735,21,8-/span>,True,


In [26]:
sentences_df = sentences_df[sentences_df["cleaned_sentence"] != ""]

In [27]:
sentences_df.to_csv("/data/semeval-2020/task-11/processed/sentence_dataset_v2.csv",index=False)